In [46]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [45]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 1024
n_epochs = 200

In [22]:
df = pd.read_csv('options-df.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['date', 'exdate', 'impl_volatility'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [23]:
call_df.head()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price
0,600.0,24.7500,25.7500,150,5633,72,5.19,617.7
14,600.0,55.7500,56.2500,0,761,352,5.19,617.7
18,610.0,17.6250,18.3750,50,1515,72,5.19,617.7
19,675.0,33.7500,34.7500,0,0,716,5.17,617.7
20,665.0,0.4375,0.6875,0,421,72,5.19,617.7


In [24]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        call_df[['best_bid', 'best_offer']],
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                    put_df[['best_bid', 'best_offer']],
                                                                    test_size=0.01, random_state=42)

In [50]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(LeakyReLU())

model.add(Dense(2, activation='relu'))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 400)               2800      
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 400)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 400)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 400)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 400)               160400    
__________

In [52]:
model.fit(call_X_train, call_y_train, 
          batch_size=n_batch, epochs=n_epochs, 
          validation_data=(call_X_test, call_y_test),
          callbacks=[TensorBoard()],
          verbose=1)

Train on 5053632 samples, validate on 51047 samples
Epoch 1/200
5053632/5053632 [==============================] - 223s 44us/step - loss: 72933.1162 - val_loss: 72543.9143
Epoch 2/200
5053632/5053632 [==============================] - 216s 43us/step - loss: 25807.4910 - val_loss: 119.9612
Epoch 3/200
5053632/5053632 [==============================] - 217s 43us/step - loss: 120.8947 - val_loss: 102.9109
Epoch 4/200
 192512/5053632 [>.............................] - ETA: 3:34 - loss: 108.9445- ETA: 3:

KeyboardInterrupt: 